In [1]:
using LinearAlgebra
using Statistics
using Distributions
using SpecialFunctions
using Combinatorics
using Plots

In [2]:
include("./graph_decomposition.jl")
include("./normalising_constant_mc.jl")
include("./normalising_constant_laplace.jl")
include("./helperfunctions.jl")
include("particle_MCMC.jl")

logIno (generic function with 1 method)

# 1. Generate data (Y) with no changepoints

In [3]:
T = 100
n_nodes = 10
mean = zeros(n_nodes)
#covariance matrix - precision matrix is its inverse which is also identity
Id = Matrix(1.0I, n_nodes, n_nodes)

10×10 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [4]:
distr = MvNormal(mean, Id)

FullNormal(
dim: 10
μ: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]
)


Generate data $y_1, y_2, ... , y_T$ from the multivariate normal likelihood

In [5]:
y = zeros(T, n_nodes)
for i in 1:T
    y[i,1:10] = rand(distr)
end

In [6]:
transpose(y)*y

10×10 Array{Float64,2}:
 105.923     -13.1658      3.79637  …  -11.8535    -6.44875  -16.7243
 -13.1658    131.49       -5.676         2.04109    9.81535    2.4661
   3.79637    -5.676     100.421        -7.27127  -16.3168     3.42619
  13.805      -6.17451     6.73558       8.75161    6.30304   11.813
  10.5361    -10.1541      6.51194       8.91187   -1.30286   -5.35676
   7.3698      0.290562   -7.80671  …    2.0849     9.2123    -0.38058
   0.775614   -3.5845      9.27696      -3.27041    3.20851   -2.74776
 -11.8535      2.04109    -7.27127      81.3915     7.90735    0.43624
  -6.44875     9.81535   -16.3168        7.90735   84.1723   -18.209
 -16.7243      2.4661      3.42619       0.43624  -18.209     84.4608

In [7]:
inv((transpose(y)*y)/100)

10×10 Array{Float64,2}:
  1.07657      0.0693635   0.00262195  …   0.148126     0.265988
  0.0693635    0.784938    0.0171246      -0.0942444   -0.0314997
  0.00262195   0.0171246   1.0601          0.200731     0.00585443
 -0.230575     0.0509773  -0.0950115      -0.200457    -0.258572
 -0.117905     0.0752959  -0.0647198       0.00612495   0.0613018
 -0.156327     0.0175273   0.056854    …  -0.163475    -0.0815873
 -0.00369656   0.0280234  -0.105409       -0.0672573    0.00894503
  0.181024    -0.0117764   0.0867361      -0.0703115    0.0228991
  0.148126    -0.0942444   0.200731        1.36594      0.344286
  0.265988    -0.0314997   0.00585443      0.344286     1.35141

# 2. Run particle MCMC

In [8]:
#parameters – 200 iterations and 100 particles

hyper = hyper_pars(10, 100, 1.0, 1.0, 3.0, 1.0/3.0, 0.5)
#(p = 10, Tau = 100, w = 1, z = 1, delta = 3, tau = 1/3, theta =0.5 )
parPF = PF_pars(100, 0.7, 0.001, 3, 1, 100, false)
#(N, ESS0, TOL, nE, nM, num_it, Laplace)
#changed N from 10 to 1, again to 1000
parMCMC = MCMC_pars(200, 0.5)
# (M, pB)

MCMC_pars(200, 0.5)

In [9]:
(changepoints, graphs, accept) = pseudo_MCMC(nothing, hyper, y, parMCMC, parPF)

Sc = nothing
P(Y|Sc) = 0.0
Graph = [0.0 1.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
proposal Spr = [46]
proposal type = Birth
P(Y|Spr) =0.0
acceptance probability = 5.00971829306495e-35
rejected
proposal Spr = [27]
proposal type = Birth
P(Y|Spr) =0.0
acceptance probability = 1.2000123491670892e-32
rejected
proposal Spr = [51]
proposal type = Birth
P(Y|Spr) =0.0
acceptance probability = 5.582882114494505e-35
rejected
proposal Spr = [38]
proposal type = Birth
P(Y|Spr) =0.0
acceptance probability = 1.8043399198773517e-36
rejected
proposal Spr = [28]
proposal type = Birth
P(Y|Spr) =0.0
acceptance probability = 4.408243465088027e-36
re

LoadError: InterruptException:

In [8]:
changepoints

201-element Array{Any,1}:
 nothing
 [33]
 [33]
 [33]
 [36]
 [36]
 [36]
 [36]
 [36]
 Any[36, 66]
 Any[14, 36]
 Any[14, 36]
 Any[14, 36]
 ⋮
 Any[14, 30, 51, 65, 85]
 Any[14, 30, 51, 65, 85]
 Any[14, 30, 51, 65, 85]
 Any[14, 30, 51, 65, 85]
 Any[14, 30, 51, 65, 85]
 Any[14, 30, 51, 65, 85]
 Any[14, 30, 51, 65, 85]
 Any[14, 30, 51, 65, 85]
 Any[14, 30, 51, 65, 85]
 Any[14, 30, 51, 65, 85]
 Any[14, 30, 51, 65, 85]
 Any[14, 30, 51, 65, 85]

In [188]:
#parameters – 100 iterations and 500 particles

hyper = hyper_pars(10, 100, 1.0, 1.0, 3.0, 1.0/3.0, 0.5)
#(p = 10, Tau = 100, w = 1, z = 1, delta = 3, tau = 1/3, theta =0.5 )
parPF = PF_pars(500, 0.7, 0.001, 3, 1, 10, true)
#(N, ESS0, TOL, nE, nM, num_it, Laplace)
#changed N from 10 to 1, again to 1000
parMCMC = MCMC_pars(100, 0.5)
# (M, pB)

MCMC_pars(100, 0.5)

In [189]:
(changepoints, graphs, accept) = pseudo_MCMC(nothing, hyper, y, parMCMC, parPF)

Sc = nothing
P(Y|Sc) = 1.0
Graph = [0.0 0.0 1.0 1.0 0.0 1.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 1.0 1.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
proposal Spr = [30]
proposal type = Birth
P(Y|Spr) =1.00000066879643
acceptance probability = 0.25000016719910734
rejected
proposal Spr = [16]
proposal type = Birth
P(Y|Spr) =1.0
acceptance probability = 0.24999999999999994
rejected
proposal Spr = [41]
proposal type = Birth
P(Y|Spr) =1.0
acceptance probability = 0.24999999999999994
accepted
proposal Spr = [28]
proposal type = Move
P(Y|Spr) =1.0
acceptance probability = 1.0
accepted
proposal Spr = Any[28, 75]
proposal type = Birth
P(Y|Spr) =1.000000000000068
acceptance probability = 0.48065476190479

P(Y|Spr) =1.0
acceptance probability = 0.24999999999999994
accepted
proposal Spr = [41]
proposal type = Move
P(Y|Spr) =1.0001836986602735
acceptance probability = 1.0394065888038142
accepted
proposal Spr = Any[41, 87]
proposal type = Birth
P(Y|Spr) =1.0000000291800901
acceptance probability = 0.48056649650872985
rejected
proposal Spr = [52]
proposal type = Move
P(Y|Spr) =1.0000000000000948
acceptance probability = 0.9998163350788212
accepted
proposal Spr = Any[52, 81]
proposal type = Birth
P(Y|Spr) =1.000000000144497
acceptance probability = 0.4806547619741693
rejected
proposal Spr = [59]
proposal type = Move
P(Y|Spr) =1.0
acceptance probability = 0.999999999999905
accepted
proposal Spr = Any[]
proposal type = Death
P(Y|Spr) =1.0
acceptance probability = 4.000000000000001
accepted
proposal Spr = [64]
proposal type = Birth
P(Y|Spr) =1.0
acceptance probability = 0.24999999999999994
accepted
proposal Spr = Any[46, 64]
proposal type = Birth
P(Y|Spr) =1.0023577819866218
acceptance probabili

(Any[nothing, nothing, nothing, [41], [28], [28], [66], Any[], Any[], Any[]  …  Any[15, 34], Any[34, 56], Any[20, 34, 56], Any[20, 34, 65], Any[20, 65], Any[20, 41], Any[20, 41, 62], Any[20, 41, 70], Any[20, 70], Any[35, 70]], Any[[0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0]

[1.0 1.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0]

[0.0 0.0 … 1.0 0.0], [0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0]

[1.0 1.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0]

[0.0 0.0 … 1.0 0.0], [0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0]

[1.0 1.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0]

[0.0 0.0 … 1.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0;

In [190]:
changepoints
#[140:160]

101-element Array{Any,1}:
 nothing
 nothing
 nothing
 [41]
 [28]
 [28]
 [66]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 [65]
 ⋮
 [45]
 [34]
 Any[15, 34]
 Any[34, 56]
 Any[20, 34, 56]
 Any[20, 34, 65]
 Any[20, 65]
 Any[20, 41]
 Any[20, 41, 62]
 Any[20, 41, 70]
 Any[20, 70]
 Any[35, 70]

In [192]:
changepoints[60:80]

21-element Array{Any,1}:
 Any[]
 Any[]
 Any[]
 Any[]
 [23]
 [41]
 [41]
 [52]
 [52]
 [59]
 Any[]
 [64]
 [64]
 [30]
 Any[]
 Any[]
 [16]
 [38]
 Any[]
 [14]
 [14]

In [185]:
changepoints[180:200]

21-element Array{Any,1}:
 Any[]
 [14]
 [14]
 [75]
 [75]
 [42]
 Any[15, 42]
 Any[15, 88]
 Any[88]
 [68]
 Any[]
 Any[]
 [31]
 [23]
 Any[23, 80]
 Any[38, 80]
 Any[24, 38, 80]
 Any[17, 38, 80]
 Any[17, 38]
 Any[38, 84]
 Any[84]

In [195]:
G = graphs[101] #graph for m in [1,101] corresponding to mcmc iteration, here m = 101
particle101_graph_2 = G[2,:,:]


10×10 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0
 0.0  0.0  0.0  0.0  1.0  0.0  1.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [196]:
particle101_graph_1 = G[1,:,:]

10×10 Array{Float64,2}:
 0.0  0.0  1.0  0.0  0.0  1.0  0.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0  1.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  1.0  0.0  1.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

# 3. Generate data with 3 changepoints

In [10]:
T = 100
n_nodes = 10
mean = zeros(n_nodes)
#covariance matrix - precision matrix is its inverse which is also identity
Id = Matrix(1.0I, n_nodes, n_nodes)

10×10 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [11]:
distr1 = MvNormal(mean, Id)

FullNormal(
dim: 10
μ: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]
)


In [12]:
Id2c = Id

Id2c[3,4] = 1/3
Id2c[4,3] = 1/3
Id2c[6,7] = 1
Id2c[7,6] = 1
Id2c[9,8] = 1
Id2c[8,9] = 1
Id2c[6,8] = 1/4
Id2c[8,6] = 1/4


0.25

In [13]:
Id2 = Id2c*transpose(Id2c)

10×10 Array{Float64,2}:
 1.0  0.0  0.0       0.0       0.0  0.0     0.0   0.0     0.0   0.0
 0.0  1.0  0.0       0.0       0.0  0.0     0.0   0.0     0.0   0.0
 0.0  0.0  1.11111   0.666667  0.0  0.0     0.0   0.0     0.0   0.0
 0.0  0.0  0.666667  1.11111   0.0  0.0     0.0   0.0     0.0   0.0
 0.0  0.0  0.0       0.0       1.0  0.0     0.0   0.0     0.0   0.0
 0.0  0.0  0.0       0.0       0.0  2.0625  2.0   0.5     0.25  0.0
 0.0  0.0  0.0       0.0       0.0  2.0     2.0   0.25    0.0   0.0
 0.0  0.0  0.0       0.0       0.0  0.5     0.25  2.0625  2.0   0.0
 0.0  0.0  0.0       0.0       0.0  0.25    0.0   2.0     2.0   0.0
 0.0  0.0  0.0       0.0       0.0  0.0     0.0   0.0     0.0   1.0

In [14]:
distr2 = MvNormal(mean, Id2)

FullNormal(
dim: 10
μ: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 2.0 0.0; 0.0 0.0 … 0.0 1.0]
)


In [15]:
Id3c = Id2c

Id3c[3,4] = 0
Id3c[4,3] = 0
Id3c[5,7] = 1/2
Id3c[7,5] = 1/2
Id3c[1,8] = 1
Id3c[8,1] = 1
Id3c[2,8] = 1/3
Id3c[8,2] = 1/3
Id3c[3,8] = 1
Id3c[8,3] = 1

1

In [16]:
Id3 = Id3c*transpose(Id3c)

10×10 Array{Float64,2}:
 2.0       0.333333   1.0       0.0  0.0   …  0.0   2.0       1.0       0.0
 0.333333  1.11111    0.333333  0.0  0.0      0.0   0.666667  0.333333  0.0
 1.0       0.333333   2.0       0.0  0.0      0.0   2.0       1.0       0.0
 0.0       0.0        0.0       1.0  0.0      0.0   0.0       0.0       0.0
 0.0       0.0        0.0       0.0  1.25     1.0   0.0       0.0       0.0
 0.25      0.0833333  0.25      0.0  0.5   …  2.0   0.5       0.25      0.0
 0.0       0.0        0.0       0.0  1.0      2.25  0.25      0.0       0.0
 2.0       0.666667   2.0       0.0  0.0      0.25  4.17361   2.0       0.0
 1.0       0.333333   1.0       0.0  0.0      0.0   2.0       2.0       0.0
 0.0       0.0        0.0       0.0  0.0      0.0   0.0       0.0       1.0

In [17]:
distr3 = MvNormal(mean, Id3)

FullNormal(
dim: 10
μ: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [2.0 0.3333333333333333 … 1.0 0.0; 0.3333333333333333 1.1111111111111112 … 0.3333333333333333 0.0; … ; 1.0 0.3333333333333333 … 2.0 0.0; 0.0 0.0 … 0.0 1.0]
)


In [18]:
Id4c = Id3c

Id4c[3,1] = 5
Id4c[1,3] = 5
Id4c[2,1] = 1
Id4c[1,2] = 1
Id4c[5,1] = 1
Id4c[1,5] = 1
Id4c[3,7] = 2
Id4c[7,3] = 2
Id4c[1,8] = 1
Id4c[8,1] = 1
Id4c[2,8] = 0
Id4c[8,2] = 0
Id4c[3,8] = 0
Id4c[8,3] = 0

0

In [19]:
Id4 = Id4c*transpose(Id4c)

10×10 Array{Float64,2}:
 29.0   2.0  10.0  0.0  2.0   0.25    10.5   2.0     1.0   0.0
  2.0   2.0   5.0  0.0  1.0   0.0      0.0   1.0     0.0   0.0
 10.0   5.0  30.0  0.0  6.0   2.0      4.0   5.0     0.0   0.0
  0.0   0.0   0.0  1.0  0.0   0.0      0.0   0.0     0.0   0.0
  2.0   1.0   6.0  0.0  2.25  0.5      1.0   1.0     0.0   0.0
  0.25  0.0   2.0  0.0  0.5   2.0625   2.0   0.5     0.25  0.0
 10.5   0.0   4.0  0.0  1.0   2.0      6.25  0.25    0.0   0.0
  2.0   1.0   5.0  0.0  1.0   0.5      0.25  3.0625  2.0   0.0
  1.0   0.0   0.0  0.0  0.0   0.25     0.0   2.0     2.0   0.0
  0.0   0.0   0.0  0.0  0.0   0.0      0.0   0.0     0.0   1.0

In [20]:
distr4 = MvNormal(mean, Id4)

FullNormal(
dim: 10
μ: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [29.0 2.0 … 1.0 0.0; 2.0 2.0 … 0.0 0.0; … ; 1.0 0.0 … 2.0 0.0; 0.0 0.0 … 0.0 1.0]
)


In [22]:
y2 = zeros(T, n_nodes)
for i in 1:T
    if i < 25
        y2[i,1:10] = rand(distr)
    elseif i < 50
        y2[i,1:10] = rand(distr2)
    elseif i < 75
        y2[i,1:10] = rand(distr3)
    else 
        y2[i,1:10] = rand(distr4)
    end
end

# 4. Run particle MCMC for data with 4 changepoints

In [23]:
#parameters

hyper = hyper_pars(10, 100, 1.0, 1.0, 3.0, 1.0/3.0, 0.5)
#(p = 10, Tau = 100, w = 1, z = 1, delta = 3, tau = 1/3, theta =0.5 )
parPF = PF_pars(100, 0.7, 0.001, 3, 1, 100, false)
#(N, ESS0, TOL, nE, nM, num_it, Laplace)
#changed N from 10 to 1, again to 1000
parMCMC = MCMC_pars(200, 0.5)
# (M, pB)

MCMC_pars(200, 0.5)

In [32]:
Spr = [50, 80]
Y = y2
pair = make_pairs(hyper)
temps = test_logl_est(Spr, hyper, Y, parPF, pair)

3-element Array{Any,1}:
 [0.0693359375, 0.13931751251220703, 0.2275710878893733, 0.33091362788854894, 0.4400322061528188, 0.6002573659157329, 0.7728025263310122, 0.9103634967165322]
 [0.0439453125, 0.12143802642822266, 0.232116243802011, 0.3543477401499331, 0.48612637521698776, 0.5980344009265695, 0.7134424928480426, 0.8181031448742457, 0.9154463837501376]
 [0.0166015625, 0.05021381378173828, 0.09195246454328299, 0.14072454896722775, 0.19191185610882844, 0.2605677823964573, 0.31761382262173066, 0.3782555629942136, 0.43593693166174263, 0.5025889153228063, 0.5637937948826954, 0.610651883323031, 0.6616017345288061, 0.7138155293964318, 0.7702698878553389, 0.8303945656431994, 0.8893589549313059, 0.9611027575930373]

In [33]:
(logl_pr, G_pr) = logl_est(Spr, hyper, Y, parPF, temps, pair)

(-1600.7082477828558, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 1.0 1.0 … 0.0 0.0; 1.0 1.0 … 0.0 0.0]

[1.0 1.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 1.0 1.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0]

[1.0 1.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; 1.0 1.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; 1.0 1.0 … 0.0 0.0]

[1.0 1.0 … 0.0 0.0; 1.0 1.0 … 1.0 0.0; 0.0 0.0 … 1.0 0.0])

In [25]:
logl_pr

-1438.7150025684562

In [24]:
(changepoints, graphs, accept) = pseudo_MCMC(nothing, hyper, y2, parMCMC, parPF)

Sc = nothing
P(Y|Sc) = 0.0
Graph = [0.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0 1.0; 0.0 0.0 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 1.0 0.0 1.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
proposal Spr = [56]
proposal type = Birth
P(Y|Spr) =0.0
acceptance probability = 3.786028602912884e9
accepted
proposal Spr = [43]
proposal type = Move
P(Y|Spr) =0.0
acceptance probability = 1.0242249315025374e-9
rejected
proposal Spr = Any[20, 56]
proposal type = Birth
P(Y|Spr) =0.0
acceptance probability = 2.822617677520882e-21
rejected
proposal Spr = [28]
proposal type = Move
P(Y|Spr) =0.0
acceptance probability = 0.00018450085984617534
rejected
proposal Spr = Any[]
proposal type = Death
P(Y|Spr) =0.0
acceptance probability = 1.1696457225269763e-

LoadError: InterruptException:

In [172]:
changepoints[140:160]

21-element Array{Any,1}:
 Any[]
 [31]
 Any[]
 [75]
 [75]
 [44]
 Any[]
 [78]
 [78]
 [82]
 Any[]
 Any[]
 Any[]
 Any[]
 [18]
 [38]
 [38]
 [72]
 Any[]
 [32]
 [32]

In [171]:
changepoints[160:180]

21-element Array{Any,1}:
 [32]
 [66]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 [17]
 [17]
 [30]
 Any[]
 Any[]
 [58]

In [170]:
changepoints[180:201]

22-element Array{Any,1}:
 [58]
 [65]
 [65]
 [43]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 Any[]
 [37]
 [55]
 Any[26, 55]
 Any[26, 50]
 Any[26, 50]
 Any[50, 82]
 Any[82]
 [34]
 Any[]
 Any[]
 [45]
 [51]

In [173]:
(graphs[1])[1,:,:]

10×10 Array{Float64,2}:
 0.0  1.0  1.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0
 0.0  0.0  1.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

# Compute marginal probabilities of edge inclusion

In [56]:
#compute marginal probabilities of edge inclusion
#M = parMCMC.M
#N = parPF.N
#pairs = make_pairs(hyper)
#probs = zeros(45)

#G0 = zeros(10,10)

#for m in 1:(M+1) 
#    gm = graphs[m]
#    for n in 1:N
#        g0 = gm[1,n,:,:]
#        G0 = G0 .+ g0  
#    end
#end

#G0 / ((M+1)*N)



# Run logl_est to get graphs given changepoints for no changepoints

In [57]:
pair = make_pairs(hyper)
temp = test_logl_est(nothing, hyper, y, parPF, pair)
(logpYS, graphs) = logl_est(nothing, hyper, y, parPF, temp, pair)

(8.199181265175042e-8, [0.0 0.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0]

[1.0 1.0 … 0.0 0.0]

[1.0 0.0 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0]

[0.0 1.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0]

[0.0 1.0 … 1.0 0.0])

In [59]:
#marginal probability of edge inclusion
#graphs
#graphs[1,10,:,:]
#G0 = zeros(10,10)


#for n in 1:N
#    g0 = graphs[1,n,:,:]
#    G0 = G0 .+ g0  
#end


#G0 / N